In [1]:
import sys
sys.path.append("../")
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import simpleland

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Linux (64bit) [/home/brandyn/anaconda3/envs/simpleland/lib/python3.7/site-packages/pymunk/libchipmunk.so]


In [3]:
from simpleland import runner

In [4]:
from simpleland.runner import get_game_def, get_player_def
import threading

import argparse
import json
import logging
import math
import socketserver
import struct
import threading
import lz4.frame
from pyinstrument import Profiler
from simpleland.client import GameClient

from simpleland.config import GameDef, PlayerDefinition, ServerConfig
from simpleland.content import Content
from simpleland.common import StateDecoder, StateEncoder


from simpleland.registry import load_game_content, load_game_def
from simpleland.renderer import Renderer
from simpleland.utils import gen_id
import traceback
from simpleland import gamectx
from simpleland.server import GameUDPServer, UDPHandler 

In [5]:
rawargs = "--game_id=survival_grid --game_tick_rate=60 --disable_sound --tile_size=128 --enable_server --enable_client".split(" ")

In [6]:
args = runner.get_arguments(rawargs)

In [7]:
if not args.enable_server and not args.enable_client and not args.remote_client:
        args.enable_client = True

if args.enable_server and args.enable_client and args.remote_client:
    print("Error: Server and Remote Client cannot be started from the same process. Please run seperately.")
    exit(1)

game_def = get_game_def(
    game_id=args.game_id,
    enable_server=args.enable_server,
    remote_client=args.remote_client,
    port=args.port,
    game_tick_rate=args.game_tick_rate,
    sim_timestep=args.sim_timestep,
    content_overrides = json.loads(args.content_overrides)
)

# Get resolution
if args.enable_client and args.resolution == 'f':
    import pygame
    pygame.init()
    infoObject = pygame.display.Info()
    resolution = (infoObject.current_w, infoObject.current_h)
else:
    res_string = args.resolution.split("x")
    resolution = (int(res_string[0]), int(res_string[1]))

player_def = get_player_def(
    enable_client=args.enable_client,
    client_id=args.client_id,
    remote_client=args.remote_client,
    hostname=args.hostname,
    port=args.port,
    disable_textures=args.disable_textures,
    render_shapes=args.render_shapes,
    resolution=resolution,
    fps=args.fps,
    player_type=args.player_type,
    tile_size=args.tile_size,
    debug_render_bodies = args.debug_render_bodies,
    view_type = args.view_type,
    sound_enabled= not args.disable_sound
)

content: Content = load_game_content(game_def)

gamectx.initialize(
    game_def,
    content=content)

if player_def.client_config.enabled:
    renderer = Renderer(
        config = player_def.renderer_config,
        asset_bundle=content.get_asset_bundle())

    client = GameClient(
        renderer=renderer,
        config=player_def.client_config)

    gamectx.add_local_client(client)

server = None
import signal
import sys
def graceful_exit(signum=None, frame=None):
    print("Shutting down")
    if game_def.server_config.enabled:
        # server.shutdown()
        server.server_close()

    if args.enable_profiler:
        profiler.stop()
        print(profiler.output_text(unicode=True, color=True))
    exit()
signal.signal(signal.SIGINT, graceful_exit)

try:
    if game_def.server_config.enabled:

        server = GameUDPServer(
            conn=(game_def.server_config.hostname, game_def.server_config.port),
            config=game_def.server_config)

        server_thread = threading.Thread(target=server.serve_forever)
        server_thread.daemon = True
        server_thread.start()
        print("Server started at {} port {}".format(game_def.server_config.hostname, game_def.server_config.port))
except (Exception,KeyboardInterrupt) as e:
    print(traceback.format_exc())
    print(e)


grid
USING STEP CLOCK
Loading Game Content.
Server started at 0.0.0.0 port 10001


In [8]:
done = True


In [ ]:
while gamectx.state == "RUNNING": 
    if done:
        gamectx.content.reset()
    gamectx.process_client_step()
    gamectx.run_step()
    gamectx.render_client_step()
    for player in gamectx.player_manager.players_map.values():
        # print(f"Player: {player.get_id()} {player.get_object_id()}")
        observation, reward, done, info = gamectx.content.get_step_info(player)
    if gamectx.config.wait_for_user_input:
        # print(f"Player: {player.get_id()} {player.get_object_id()}")
        # print(observation)
        print(reward)
        print(done)
        print(info)
        print("----------")


    gamectx.wait_for_input()

request_info {'client_id': '2', 'last_latency_ms': None, 'snapshots_received': [], 'player_type': 0, 'message': 'UPDATE'}
request_info {'client_id': '2', 'last_latency_ms': None, 'snapshots_received': [1], 'player_type': 0, 'message': 'UPDATE'}
request_info {'client_id': '2', 'last_latency_ms': None, 'snapshots_received': [1], 'player_type': 0, 'message': 'UPDATE'}
request_info {'client_id': '2', 'last_latency_ms': None, 'snapshots_received': [1], 'player_type': 0, 'message': 'UPDATE'}
request_info {'client_id': '2', 'last_latency_ms': None, 'snapshots_received': [1], 'player_type': 0, 'message': 'UPDATE'}
request_info {'client_id': '2', 'last_latency_ms': None, 'snapshots_received': [1], 'player_type': 0, 'message': 'UPDATE'}
request_info {'client_id': '2', 'last_latency_ms': None, 'snapshots_received': [1], 'player_type': 0, 'message': 'UPDATE'}
request_info {'client_id': '2', 'last_latency_ms': None, 'snapshots_received': [1], 'player_type': 0, 'message': 'UPDATE'}
request_info {'cl

In [ ]:
# for i in range(10):
#     if done:
#         gamectx.content.reset()
#     gamectx.process_client_step()
#     gamectx.run_step()
#     gamectx.render_client_step()
#     for player in gamectx.player_manager.players_map.values():
#         # print(f"Player: {player.get_id()} {player.get_object_id()}")
#         observation, reward, done, info = gamectx.content.get_step_info(player)
#     if gamectx.config.wait_for_user_input:
#         # print(f"Player: {player.get_id()} {player.get_object_id()}")
#         # print(observation)
#         print(reward)
#         print(done)
#         print(info)
#         print("----------")


#     gamectx.wait_for_input()